In [30]:
import numpy as np
import pandas as pd
from pandas_datareader import data as wb
import matplotlib.pyplot as plt

In [31]:
## BEI.DE = ação alemã da PeG exportando infos do yahoo finance
tickers = ['PG', 'BEI.DE']

secData = pd.DataFrame()

for t in tickers:
    secData[t] = wb.DataReader(t, data_source = 'yahoo', start='2007-1-1')['Adj Close']

In [32]:
secData.tail()

,PG,BEI.DE
Date,,
2021-12-21,157.759995,88.699997
2021-12-22,159.320007,90.540001
2021-12-23,160.100006,90.019997
2021-12-27,161.970001,90.080002
2021-12-28,162.839996,90.720001


In [33]:
secReturns = np.log(secData / secData.shift(1))

In [34]:
secReturns

,PG,BEI.DE
Date,,
2007-01-03,NaN,NaN
2007-01-04,-0.007621,0.006544
2007-01-05,-0.008624,-0.020772
2007-01-08,0.002202,0.000202
2007-01-09,-0.002518,-0.022858
...,...,...
2021-12-21,-0.000634,0.001354
2021-12-22,0.009840,0.020532
2021-12-23,0.004884,-0.005760


# Cálculo PeG

In [35]:
## média diaria
secReturns['PG'].mean()

0.00036175700217756356

In [36]:
## media anual
secReturns['PG'].mean() * 250

0.09043925054439089

In [37]:
## std() = desvio padrao
secReturns['PG'].std()

0.011715930281222621

In [38]:
##formula do desvio padrao
secReturns['PG'].std() * 250 ** 0.5

0.18524512298200266

# Beiersdorf

In [39]:
## média diaria
secReturns['BEI.DE'].mean()

0.00019580279789853456

In [40]:
## media anual
secReturns['BEI.DE'].mean() * 250

0.04895069947463364

In [41]:
## std() = desvio padrao
secReturns['BEI.DE'].std()

0.013698939167385175

In [42]:
##formula do desvio padrao
secReturns['BEI.DE'].std() * 250 ** 0.5

0.21659924648513879

In [43]:
##imprimindo as 2 eq das medias anualizadas para comparar
print(secReturns['PG'].mean() * 250)
print(secReturns['BEI.DE'].mean() * 250)

0.09043925054439089
0.04895069947463364


In [44]:
##criando com formado de array (numero de colchete = numero de ativos) para outra forma de visualização das médias
secReturns[['PG','BEI.DE']].mean() * 250

PG        0.090439
BEI.DE    0.048951
dtype: float64

In [45]:
##criando com formado de array (numero de colchete = numero de ativos) para outra forma de visualização dos desvios
secReturns[['PG','BEI.DE']].std() * 250 ** 0.5

PG        0.185245
BEI.DE    0.216599
dtype: float64

In [46]:
##Resumindo, a PeG tem mais retorno (maior média) com menos risco (menor desvio padrão)

# Covariancia e Correlacao

In [47]:
#.var() calcula a variancia

In [48]:
PGVar = secReturns['PG'].var()
PGVar

0.00013726302235446915

In [49]:
BEIVar = secReturns['BEI.DE'].var()
BEIVar

0.00018766093431171963

In [50]:
#anual
PGVarAnual = secReturns['PG'].var() * 250
PGVarAnual

0.03431575558861729

In [51]:
#anual
BEIVarAnual = secReturns['BEI.DE'].var() * 250
BEIVarAnual

0.046915233577929906

In [52]:
#.cov() calcula a covariancia de pares de colunas

In [53]:
COVMatriz = secReturns.cov()
COVMatriz

,PG,BEI.DE
PG,0.000137,0.000043
BEI.DE,0.000043,0.000188


In [54]:
COVMatrizAnual = secReturns.cov() * 250
COVMatrizAnual

,PG,BEI.DE
PG,0.034316,0.010772
BEI.DE,0.010772,0.046915


In [55]:
# .corr() calcula a correlação de pares de colunas
CorrMatriz = secReturns.corr()
CorrMatriz

,PG,BEI.DE
PG,1.000000,0.268149
BEI.DE,0.268149,1.000000


# Calculando Risco de uma carteira

In [56]:
# ativos com pesos iguais
weights = np.array([0.5,0.5])

In [58]:
#calculo a variancia da carteira por multiplicação de matrizes, .T é a matriz transposta
portfolioVar = np.dot(weights.T, np.dot(secReturns.cov() * 250, weights))
portfolioVar

0.02569393895104645

In [61]:
#calculando a volatilidade da carteira, lembrando que .dot faz multiplicação da matriz/array
portfolioVol = (np.dot(weights.T, np.dot(secReturns.cov() * 250, weights))) ** 0.5
portfolioVol

0.16029329041181495

In [70]:
print (str(round(portfolioVol, 5) * 100) + '%')

16.029%


# Avaliando a diversificação da carteira

In [71]:
# ativos com pesos iguais
weights = np.array([0.5,0.5])

#### Risco sistemático => risco diversificável = variância da carteira - variancia anual ponderada

In [72]:
float(PGVarAnual)

0.03431575558861729

In [73]:
#calculando a diversificação
dr =portfolioVar - (weights[0] **2*PGVarAnual) - (weights[1] **2*PGVarAnual)
dr

0.008536061156737803

In [74]:
#valor acima mostra que é menor que 1%

#### Risco não sistemático

In [75]:
drNot = portfolioVar - dr
drNot

0.01715787779430865